## ***SACHAROMYCES CEREVISIAE*** OPTIMIZATION OUTPUT DATA PROCESSING

Since we've runned 10 Gene Over/Under expression optimization problem we've quite a lot of results to carefully interpret. We'll be using pandas (a powerful python library to handle data science preprocessing stages). <br> First we'll merge all 10 .csv file into 1 .csv file to better understand our data and preform various preprocessing steps to gather only the best solutions possible for our problem.

In [2]:
import os
import glob
import pandas as pd

# pandas has a set_option method that let you costumize the behaviour and display of dataframes
# here the set_option method is set to display the full contents of our dataframe
# pd.set_option("display.max_rows", None)
# pd.options.display.max_rows = 15

# setting the path to join multiple files 
files = os.path.join(r'C:\Users\geral\Desktop\BIOINF_PROJECT\2-PEopt\src\outputGOU', "*.csv")

# return a list of the merged files
files = glob.glob(files)

# using pandas methods concat and read_csv to join files
df = pd.concat([pd.read_csv(f, sep=";", index_col=False) for f in files], ignore_index=True)
print(f"This dataframe has: {df.shape[0]} rows and {df.shape[1]} columns.")

# pandas dataframe manipulation to drop unnecessary columns (columns only with NaN values)
df.drop(["Unnamed: 3", "solution"], axis=1, inplace=True)
print(f"This dataframe has: {df.shape[0]} rows and {df.shape[1]} columns after manipulation.")

# change column names
df.rename(columns={"Unnamed: 4": "Genes", "Unnamed: 6": "Reactions"}, inplace=True)


This dataframe has: 189 rows and 7 columns.
This dataframe has: 189 rows and 5 columns after manipulation.


In [3]:
# cell to display part of out dataframe
display(df)

,BPCY (r_4041 * r_1589),TargetFlux r_4041 with at least 0.0 of biomass (None),WYIELD biomass: r_4041 product: r_1589,Genes,Reactions
0,0.000000,0.000000,0.000000,"{'YBR166C': 0.25, 'YNL220W': 0.125, 'YIL043C':...","{'r_0714': 0, 'r_0020': (2.6153065645050235, 1..."
1,0.000000,0.000000,0.000000,"{'YBR166C': 0.25, 'YNL220W': 0.125, 'YGL063W':...","{'r_0714': 0, 'r_0020': (2.6153065645050084, 1..."
2,0.000000,0.000000,0.000000,"{'YDR305C': 0.5, 'YGR208W': 0.25, 'YOL126C': 0...","{'r_1025': (0, 0), 'r_0714': 0, 'r_0138': (0.0..."
3,0.000000,0.000000,0.000000,"{'YDR305C': 0.5, 'YNL241C': 4, 'YGR208W': 0.25...","{'r_1025': (0, 0), 'r_0714': 0, 'r_0138': (0.0..."
4,0.000000,0.000000,0.000000,"{'YDR305C': 0.5, 'YNL241C': 4, 'YGR208W': 0.25...","{'r_1025': (0, 0), 'r_0714': 0, 'r_0138': (0.0..."
...,...,...,...,...,...
184,0.650626,0.598086,0.351609,"{'YMR267W': 2, 'YLR308W': 2, 'YNL331C': 0.25, ...","{'r_1069': 0, 'r_0468': (0, 0), 'r_1109': (0, ..."
185,0.249412,0.094355,1.269405,"{'YJR103W': 16, 'YJL121C': 4, 'YFL058W': 0.125...","{'r_0279': (2.644694430440478, 10000), 'r_0307..."
186,0.090259,0.094355,1.319482,"{'YPL262W': 2, 'YGR193C': 0.125, 'YJL121C': 2,...","{'r_0279': (2.6446944304404685, 10000), 'r_045..."
187,0.054513,0.094355,1.265827,"{'YPL262W': 2, 'YNL191W': 0, 'YGL148W': 4, 'YF...","{'r_4171': (0, 0), 'r_0279': (1.32234721522023..."


Our first step to achieve a desirable solution is to drop all rows that doesn't correlate with our objetive: Biomass > 0.1 <br> So let's drop some rows and see what we have.

In [4]:
df.drop(df[df["BPCY (r_4041 * r_1589)"] < 0.1].index, inplace=True)
print(f"This dataframe has {df.shape[0]} rows and {df.shape[1]} columns after deleting all rows with biomass inferior to 0.1.")
df.reset_index(drop=True)

This dataframe has 115 rows and 5 columns after deleting all rows with biomass inferior to 0.1.


,BPCY (r_4041 * r_1589),TargetFlux r_4041 with at least 0.0 of biomass (None),WYIELD biomass: r_4041 product: r_1589,Genes,Reactions
0,0.471155,0.189779,0.972732,"{'YGL063W': 8, 'YIL043C': 0.03125, 'YOL126C': ...","{'r_0714': 0, 'r_0020': (2.6153065645050195, 1..."
1,0.492589,0.187592,1.648812,"{'YBR166C': 0.25, 'YOL126C': 0, 'YBR249C': 8, ...","{'r_0714': 0, 'r_0020': (2.615306564505013, 10..."
2,0.470345,0.189315,0.988020,"{'YDL085W': 0.25, 'YGL065C': 0, 'YBR249C': 8, ...","{'r_4244': 0, 'r_0714': 0, 'r_4243': 0, 'r_002..."
3,0.492589,0.187592,1.794705,"{'YBR166C': 0.25, 'YDL085W': 0.25, 'Q0105': 16...","{'r_0714': 0, 'r_0020': (2.615306564505009, 10..."
4,0.492589,0.187592,1.794705,"{'YBR166C': 0.25, 'YDL085W': 0.25, 'Q0105': 16...","{'r_0714': 0, 'r_0020': (2.615306564505013, 10..."
...,...,...,...,...,...
110,0.650626,0.598086,0.351609,"{'YMR267W': 2, 'YLR308W': 2, 'YNL331C': 0.25, ...","{'r_0468': (0, 0), 'r_4250': 0, 'r_1069': 0, '..."
111,0.650626,0.598086,0.351609,"{'YMR267W': 2, 'YLR308W': 2, 'YNL331C': 0.25, ...","{'r_1069': 0, 'r_0468': (0, 0), 'r_1109': (0, ..."
112,0.650626,0.598086,0.351609,"{'YMR267W': 2, 'YLR308W': 2, 'YNL331C': 0.25, ...","{'r_1069': 0, 'r_0468': (0, 0), 'r_1109': (0, ..."
113,0.249412,0.094355,1.269405,"{'YJR103W': 16, 'YJL121C': 4, 'YFL058W': 0.125...","{'r_0279': (2.644694430440478, 10000), 'r_0307..."


In order to analyse the natural maximum 2-phenylethanol production capabilities of *Sacharomyces cerevisiae* we perform a FVA analysis on the YEAST8 model using MEWpy

In [5]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator

model = read_sbml_model("models/yeast-GEM.xml")

O2 = "r_1992" # EX_o2_e (r_1992)
GLC = "r_1714" # EX_glc__D_e (r_1714)

envcond = {
        O2: (-20.0, 100000.0), 
        GLC: (-10.0, 100000.0)
        }

PRODUCT_ID = "r_1589" # EX_2phetoh_e (r_1589)

simul = get_simulator(model, envcond=envcond)
print(simul.FVA(reactions = [PRODUCT_ID], format = 'df'))

  Reaction ID  Minimum   Maximum
0      r_1589      0.0  0.927562


However, by analysing our optimization results within our dataframe we can actually see that there are a set of modifications that increases the *Sacharomyces cerevisiae* natural production capabilities of 2-phenylethanol by almost 4 times.

In [6]:
print(f"Minimum FVA: {df['WYIELD biomass: r_4041 product: r_1589 '].min()}")
print(f"Maximum FVA: {df['WYIELD biomass: r_4041 product: r_1589 '].max()}")


Minimum FVA: 0.1517507097982077
Maximum FVA: 3.70211854292926


In [7]:
# get the row which corresponds with our maximum level of WYIELD
df.loc[df["WYIELD biomass: r_4041 product: r_1589 "] == df["WYIELD biomass: r_4041 product: r_1589 "].max()]


,BPCY (r_4041 * r_1589),TargetFlux r_4041 with at least 0.0 of biomass (None),WYIELD biomass: r_4041 product: r_1589,Genes,Reactions
128,0.311934,0.077422,3.702119,"{'YGR256W': 2, 'YDL198C': 0.0625, 'YLR359W': 0...","{'r_0151': (0, 0.08376934531786118), 'r_0152':..."


In [8]:
print(f"Genes: {df['Genes'][128]}")
print(f"Reactions: {df['Reactions'][128]}")

Genes: {'YGR256W': 2, 'YDL198C': 0.0625, 'YLR359W': 0.5, 'YCL050C': 2, 'YDR497C': 0.25, 'YNL316C': 32, 'Q0045': 0.25, 'YPR145W': 16}
Reactions: {'r_0151': (0, 0.08376934531786118), 'r_0152': (0, 0.09484195786964987), 'r_0211': (1.469947402640923, 10000), 'r_0438': (0, 19.307971581669833), 'r_0889': (7.068834785461117, 10000), 'r_0938': (3.8707169858806956, 10000), 'r_1026': (0.10382579193452601, 10000)}


Our goal now is to reduce even more our pool of possible solutions by choosing the ones that have less genetic modifications. For that we will convert the dataframe solution values to a dictionary and from there we will be able to check the ones that are more suitable within our main goal. <br>
**ast** library makes transforming a dictionary formatted string into a python dictionary object.

In [25]:
import ast

genes = [ast.literal_eval(x) for x in df["Genes"]]
genes2 = [i for i in genes if len(i) < 5]
genes2



[{'YPR060C': 16, 'YFR033C': 0.5, 'YKL060C': 0.25},
 {'YBR249C': 8, 'YKL029C': 0, 'YDR298C': 0.25, 'YDR354W': 16},
 {'YDR298C': 0.5, 'YBR218C': 2, 'YBR249C': 8, 'YKL029C': 0},
 {'YBR249C': 8, 'YKL029C': 0, 'YDR298C': 0.25, 'YJR024C': 4},
 {'YBR249C': 8, 'YKL029C': 0, 'YDR298C': 0.25},
 {'YPL028W': 0, 'YMR105C': 8, 'YNL316C': 32, 'YOR065W': 0.5},
 {'YDL205C': 0.25, 'YHR183W': 4, 'YNL316C': 8, 'YER043C': 16},
 {'YPL028W': 0, 'YNL316C': 32, 'YOR065W': 0.5},
 {'YAL054C': 2, 'YBR249C': 8, 'YHR183W': 4, 'YPL214C': 0.125},
 {'YGR256W': 2, 'YNL316C': 32, 'Q0045': 0.25},
 {'YPR145W': 32, 'YNL316C': 16, 'YGR256W': 4, 'Q0045': 0.5}]

In [18]:
import ast

reactions = [ast.literal_eval(x) for x in df["Reactions"]]
reaction2 = [i for i in reactions if len(i) < 5]
reaction2[2]

{'r_0144': (1.7631496590944593, 10000),
 'r_0557': (0, 1.9660901513046476e-07),
 'r_0889': (14.137669570922139, 10000),
 'r_0938': (0.9676792464701721, 10000)}